## Basic vocabulary with mapping from indexes to words

In [4]:
SOS_token = 0 # Start of sentence
EOS_token = 1   # End of sentence

index2words = {
    SOS_token : "SOS",
    EOS_token : "EOS"
}

words = "How are you doing ? I am doing good and you ?"
words_list = set(words.lower().split(" "))
for word in words_list:
    index2words[len(index2words)] = word
    
print(index2words)

{0: 'SOS', 1: 'EOS', 2: '?', 3: 'how', 4: 'are', 5: 'and', 6: 'doing', 7: 'you', 8: 'good', 9: 'i', 10: 'am'}


In [5]:
# Getting the reverse mapping
words2index = {v: k for k, v in index2words.items()}
words2index

{'SOS': 0,
 'EOS': 1,
 '?': 2,
 'how': 3,
 'are': 4,
 'and': 5,
 'doing': 6,
 'you': 7,
 'good': 8,
 'i': 9,
 'am': 10}

In [6]:
# Functions to convert sentence to tensor
import torch

def convert2tensors(sentecnce):
    words_list = sentecnce.lower().split(" ")  # Convert the whole sentence into list of words
    indexes = [words2index[word] for word in words_list]  # Convert the words into indexes
    #print(indexes)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)  # Convert the indexes into tensor

sentence = "How are you doing ?"
print(convert2tensors(sentence))

tensor([[3],
        [4],
        [7],
        [6],
        [2]])


### The Encoder

In [7]:
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        
    def forward(self,input_tensor):
        embedded = self.embedding(input_tensor)
        output,hidden = self.gru(embedded) # only 1 layer of GRU as of now
        return output, hidden    

In [11]:
encoder = EncoderRNN(input_size=len(index2words),hidden_size=3)

sentence = "How are you doing ?"
input_tensor = convert2tensors(sentence)
output,hidden = encoder(input_tensor)


In [12]:
output.size()

torch.Size([5, 1, 3])

In [13]:
hidden.size()

torch.Size([1, 1, 3])

In [14]:
output[:,-1]

tensor([[-0.3397,  0.3092,  0.2277],
        [ 0.2033,  0.5876,  0.2414],
        [-0.1237,  0.0848,  0.4365],
        [ 0.1201,  0.1303,  0.3473],
        [ 0.0133, -0.0310,  0.4918]], grad_fn=<SelectBackward0>)

### The Decoder

In [ ]:
import torch.nn.functional as F

MAX_LENGTH = 10

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input_tensor, hidden):
        output = self.embedding(input_tensor)
        output = F.relu(output)
        output,hidden = self.gru(output,hidden)
        
        
        
        
        